In [1]:
import pandas as pd
import collections
import math
import os

import cv2
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import math
import time
from PIL import Image 

from pandas import read_csv  
import configparser

config = configparser.ConfigParser()
config.read('configtest.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

window_size = int(config.get('Configuration', 'window_size'))

scenes=config.get('Configuration', 'scenes')
scenes = [(item.strip()) for item in scenes.split(',')]

print(scenes)

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

room=config.get('Configuration', 'room')

w = int(config.get(room, 'w'))
h = int(config.get(room, 'h'))

user_color={}
for user in users:
    user_color[user]=config.get('Colors', user)
print(user_color)


['Scene1', 'Scene2', 'Scene3', 'Scene4', 'Scene5', 'Scene6']
{'f02e': 'blue', '867f': 'red'}


In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

days=list(range(day_time(t0),day_time(tN)+1))
print(days)

i0=0
ts=list(range(t0,tN,1))
print(i0,ts)
print(len(ts))

2024-03-16 19:30:50 Init date is 1710613850 day: 19798 2024-03-16 19:30:50
2024-03-16 20:26:35 End date is 1710617195 day: 19798 2024-03-16 20:26:35
[19798]
0 [1710613850, 1710613851, 1710613852, 1710613853, 1710613854, 1710613855, 1710613856, 1710613857, 1710613858, 1710613859, 1710613860, 1710613861, 1710613862, 1710613863, 1710613864, 1710613865, 1710613866, 1710613867, 1710613868, 1710613869, 1710613870, 1710613871, 1710613872, 1710613873, 1710613874, 1710613875, 1710613876, 1710613877, 1710613878, 1710613879, 1710613880, 1710613881, 1710613882, 1710613883, 1710613884, 1710613885, 1710613886, 1710613887, 1710613888, 1710613889, 1710613890, 1710613891, 1710613892, 1710613893, 1710613894, 1710613895, 1710613896, 1710613897, 1710613898, 1710613899, 1710613900, 1710613901, 1710613902, 1710613903, 1710613904, 1710613905, 1710613906, 1710613907, 1710613908, 1710613909, 1710613910, 1710613911, 1710613912, 1710613913, 1710613914, 1710613915, 1710613916, 1710613917, 1710613918, 1710613919, 

In [3]:
scene_user = {}
lista_timestamps=[]
for d in days:
    for user in users:
        scene_user[user] = {}
        for s in scenes:
            print(d,source_folder+"/"+s+"/location_gt_"+user+"/"+str(d)+".location.tsv")
            try:
                series=read_csv(source_folder+"/"+s+"/location_gt_"+user+"/"+str(d)+".location.tsv", sep="\t",parse_dates=True, header=None, names=["time","date","time2","x","y"])
                print(d,series.size,(series["time"].iloc[0],series["time"].iloc[-1]))
                scene_user[user][s] = (series["time"].iloc[0],series["time"].iloc[-1])
                #print("test")
                print(scene_user[user][s])
                #if user=='ed9c':
                lista_timestamps.extend(list(range(scene_user[user][s][0],scene_user[user][s][1],1)))
                #print(lista_timestamps)
                #print("test2")
            except Exception as e:
                print("Not data",e)
                continue
def getScene(t0):
    for user, user_scene in scene_user.items():
        for scene, times in user_scene.items():
            #print(t0,times[0],times[1])
            if(t0>times[0]-window_size/2 and t0<times[1]+window_size/2):
                return scene
    return None

19798 ./data/livingroom/2-person-labelled//Scene1/location_gt_f02e/19798.location.tsv
19798 1960 (1710613851, 1710614183)
(1710613851, 1710614183)
19798 ./data/livingroom/2-person-labelled//Scene2/location_gt_f02e/19798.location.tsv
19798 2510 (1710614425, 1710614773)
(1710614425, 1710614773)
19798 ./data/livingroom/2-person-labelled//Scene3/location_gt_f02e/19798.location.tsv
19798 2290 (1710615299, 1710615618)
(1710615299, 1710615618)
19798 ./data/livingroom/2-person-labelled//Scene4/location_gt_f02e/19798.location.tsv
19798 2085 (1710615774, 1710616088)
(1710615774, 1710616088)
19798 ./data/livingroom/2-person-labelled//Scene5/location_gt_f02e/19798.location.tsv
19798 1660 (1710616615, 1710616905)
(1710616615, 1710616905)
19798 ./data/livingroom/2-person-labelled//Scene6/location_gt_f02e/19798.location.tsv
19798 1290 (1710617082, 1710617196)
(1710617082, 1710617196)
19798 ./data/livingroom/2-person-labelled//Scene1/location_gt_867f/19798.location.tsv
19798 1350 (1710613852, 17106141

In [4]:
#print(len(lista_timestamps))
def getFPImg(tt):
    return np.array(Image.open(data_folder+"/"+room+"/DAY_"+str(day)+"/FP."+str(tt)+".png"))/255.0

def getGTImg(tt,user):
    #try:
    return np.array(Image.open(data_folder+"/"+room+"/DAY_"+str(day)+"/GT."+user+"."+str(tt)+".png"))/255.0
    #except FileNotFoundError:
     #   print("File not found")
     #   return None

In [5]:
#ts=lista_timestamps
#print(ts)
XY0={}
rssi0={}
Y0={}

T0_user={}
T0=[]
T1=[]
for day in days:
    anchors_data={}
    for user in users:
        XY0[user]=[]
        rssi0[user]=[]
        Y0[user]=[]
        T0_user[user]=[]


        anchors_data[user]={}
        for anchor in anchors:
            print(data_folder+"/"+room+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv")
            print(user,anchor)
            series=read_csv(data_folder+"/"+room+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","d"])
            series=series.values
            anchors_data[user][anchor]=series
            

    for it,tt in enumerate(ts):
        print(it,tt)
        imgX=getFPImg(tt)
        print("imgX",imgX.shape)

        for user in users:
            
            imgY=getGTImg(tt,user)
            #if imgY is None:
            #    continue
            print("user",user,"imgY",imgY.shape)

            dist=[]
            for ixa,anchor in enumerate(anchors):
                d=anchors_data[user][anchor][it][1]
                if(d>0):
                    d=d/10.0
                if(d>1):
                    d=1
                print("\t anchor:",anchor, anchors_data[user][anchor][it], "tt:",tt)
                dist.append(d)

            dist=np.array(dist)
            print("dist:",dist)

            rssi0[user].append(dist.reshape(len(anchors),1))
            XY0[user].append(imgX[:, :, np.newaxis])
            Y0[user].append(imgY[:, :, np.newaxis])
            T0_user[user].append(it)
            T0.append(it)
            T1.append(tt)
print(T0)        
print(T1)        
print(len(T1))
T0=list(dict.fromkeys(T0))
T1=list(dict.fromkeys(T1))
print(T0)        
print(T1)
#print(T0_user)

./2p-livingroom/livingroom/DAY_19798/f02e.8171.tsv
f02e 8171
./2p-livingroom/livingroom/DAY_19798/f02e.eb38.tsv
f02e eb38
./2p-livingroom/livingroom/DAY_19798/867f.8171.tsv
867f 8171
./2p-livingroom/livingroom/DAY_19798/867f.eb38.tsv
867f eb38
0 1710613850
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061385e+09 5.14200000e+00] tt: 1710613850
	 anchor: eb38 [1.71061385e+09 3.58200000e+00] tt: 1710613850
dist: [0.5142 0.3582]
user 867f imgY (8, 8)
	 anchor: 8171 [1.71061385e+09 4.91200000e+00] tt: 1710613850
	 anchor: eb38 [1.71061385e+09 3.56800000e+00] tt: 1710613850
dist: [0.4912 0.3568]
1 1710613851
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061385e+09 5.14700000e+00] tt: 1710613851
	 anchor: eb38 [1.71061385e+09 3.69900000e+00] tt: 1710613851
dist: [0.5147 0.3699]
user 867f imgY (8, 8)
	 anchor: 8171 [1.71061385e+09 4.91200000e+00] tt: 1710613851
	 anchor: eb38 [1.71061385e+09 3.56800000e+00] tt: 1710613851
dist: [0.4912 0.3568]
2 1710613852
imgX (8, 8)
user f02e

user 867f imgY (8, 8)
	 anchor: 8171 [1.71061388e+09 6.56200000e+00] tt: 1710613876
	 anchor: eb38 [1.71061388e+09 3.39000000e+00] tt: 1710613876
dist: [0.6562 0.339 ]
27 1710613877
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061388e+09 5.40900000e+00] tt: 1710613877
	 anchor: eb38 [1.71061388e+09 3.34800000e+00] tt: 1710613877
dist: [0.5409 0.3348]
user 867f imgY (8, 8)
	 anchor: 8171 [1.71061388e+09 6.56200000e+00] tt: 1710613877
	 anchor: eb38 [1.71061388e+09 3.39000000e+00] tt: 1710613877
dist: [0.6562 0.339 ]
28 1710613878
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061388e+09 5.32000000e+00] tt: 1710613878
	 anchor: eb38 [1.71061388e+09 3.25900000e+00] tt: 1710613878
dist: [0.532  0.3259]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061388e+09 -1.00000000e+00] tt: 1710613878
	 anchor: eb38 [ 1.71061388e+09 -1.00000000e+00] tt: 1710613878
dist: [-1. -1.]
29 1710613879
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061388e+09 5.32000000e+00] tt: 1710613879


user f02e imgY (8, 8)
	 anchor: 8171 [1.71061396e+09 5.90600000e+00] tt: 1710613962
	 anchor: eb38 [1.71061396e+09 2.87100000e+00] tt: 1710613962
dist: [0.5906 0.2871]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061396e+09 -1.00000000e+00] tt: 1710613962
	 anchor: eb38 [ 1.71061396e+09 -1.00000000e+00] tt: 1710613962
dist: [-1. -1.]
113 1710613963
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061396e+09 5.89100000e+00] tt: 1710613963
	 anchor: eb38 [1.71061396e+09 3.01100000e+00] tt: 1710613963
dist: [0.5891 0.3011]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061396e+09 -1.00000000e+00] tt: 1710613963
	 anchor: eb38 [ 1.71061396e+09 -1.00000000e+00] tt: 1710613963
dist: [-1. -1.]
114 1710613964
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061396e+09 5.89100000e+00] tt: 1710613964
	 anchor: eb38 [1.71061396e+09 3.08600000e+00] tt: 1710613964
dist: [0.5891 0.3086]
user 867f imgY (8, 8)
	 anchor: 8171 [1.71061396e+09 8.03800000e+00] tt: 1710613964
	 anchor: eb38 [ 1.7106139

imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061401e+09 3.98500000e+00] tt: 1710614012
	 anchor: eb38 [1.71061401e+09 4.21400000e+00] tt: 1710614012
dist: [0.3985 0.4214]
user 867f imgY (8, 8)
	 anchor: 8171 [1.71061401e+09 6.34600000e+00] tt: 1710614012
	 anchor: eb38 [1.71061401e+09 1.35400000e+00] tt: 1710614012
dist: [0.6346 0.1354]
163 1710614013
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [ 1.71061401e+09 -1.00000000e+00] tt: 1710614013
	 anchor: eb38 [ 1.71061401e+09 -1.00000000e+00] tt: 1710614013
dist: [-1. -1.]
user 867f imgY (8, 8)
	 anchor: 8171 [1.71061401e+09 6.34600000e+00] tt: 1710614013
	 anchor: eb38 [1.71061401e+09 1.42400000e+00] tt: 1710614013
dist: [0.6346 0.1424]
164 1710614014
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [ 1.71061401e+09 -1.00000000e+00] tt: 1710614014
	 anchor: eb38 [ 1.71061401e+09 -1.00000000e+00] tt: 1710614014
dist: [-1. -1.]
user 867f imgY (8, 8)
	 anchor: 8171 [1.71061401e+09 6.51500000e+00] tt: 1710614014
	 anchor: eb38

user 867f imgY (8, 8)
	 anchor: 8171 [1.71061409e+09 4.17700000e+00] tt: 1710614088
	 anchor: eb38 [1.71061409e+09 4.38300000e+00] tt: 1710614088
dist: [0.4177 0.4383]
239 1710614089
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061409e+09 5.39500000e+00] tt: 1710614089
	 anchor: eb38 [1.71061409e+09 2.66000000e+00] tt: 1710614089
dist: [0.5395 0.266 ]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061409e+09 -1.00000000e+00] tt: 1710614089
	 anchor: eb38 [ 1.71061409e+09 -1.00000000e+00] tt: 1710614089
dist: [-1. -1.]
240 1710614090
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061409e+09 5.40400000e+00] tt: 1710614090
	 anchor: eb38 [1.71061409e+09 2.91700000e+00] tt: 1710614090
dist: [0.5404 0.2917]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061409e+09 -1.00000000e+00] tt: 1710614090
	 anchor: eb38 [ 1.71061409e+09 -1.00000000e+00] tt: 1710614090
dist: [-1. -1.]
241 1710614091
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061409e+09 5.40400000e+00] tt: 1710614091

...

imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061672e+09 4.01300000e+00] tt: 1710616717
	 anchor: eb38 [1.71061672e+09 5.24100000e+00] tt: 1710616717
dist: [0.4013 0.5241]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061672e+09 -1.00000000e+00] tt: 1710616717
	 anchor: eb38 [ 1.71061672e+09 -1.00000000e+00] tt: 1710616717
dist: [-1. -1.]
2868 1710616718
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [ 1.71061672e+09 -1.00000000e+00] tt: 1710616718
	 anchor: eb38 [ 1.71061672e+09 -1.00000000e+00] tt: 1710616718
dist: [-1. -1.]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061672e+09 -1.00000000e+00] tt: 1710616718
	 anchor: eb38 [ 1.71061672e+09 -1.00000000e+00] tt: 1710616718
dist: [-1. -1.]
2869 1710616719
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [ 1.71061672e+09 -1.00000000e+00] tt: 1710616719
	 anchor: eb38 [ 1.71061672e+09 -1.00000000e+00] tt: 1710616719
dist: [-1. -1.]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061672e+09 -1.00000000e+00] tt: 1710616719
	 anchor: eb38

user f02e imgY (8, 8)
	 anchor: 8171 [ 1.71061684e+09 -1.00000000e+00] tt: 1710616837
	 anchor: eb38 [ 1.71061684e+09 -1.00000000e+00] tt: 1710616837
dist: [-1. -1.]
user 867f imgY (8, 8)
	 anchor: 8171 [1.71061684e+09 4.85200000e+00] tt: 1710616837
	 anchor: eb38 [1.71061684e+09 3.70800000e+00] tt: 1710616837
dist: [0.4852 0.3708]
2988 1710616838
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061684e+09 4.58000000e+00] tt: 1710616838
	 anchor: eb38 [1.71061684e+09 4.58900000e+00] tt: 1710616838
dist: [0.458  0.4589]
user 867f imgY (8, 8)
	 anchor: 8171 [1.71061684e+09 4.67900000e+00] tt: 1710616838
	 anchor: eb38 [1.71061684e+09 3.60100000e+00] tt: 1710616838
dist: [0.4679 0.3601]
2989 1710616839
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061684e+09 4.58000000e+00] tt: 1710616839
	 anchor: eb38 [1.71061684e+09 4.58900000e+00] tt: 1710616839
dist: [0.458  0.4589]
user 867f imgY (8, 8)
	 anchor: 8171 [1.71061684e+09 4.67900000e+00] tt: 1710616839
	 anchor: eb38 [1.7106

user 867f imgY (8, 8)
	 anchor: 8171 [1.71061694e+09 4.79100000e+00] tt: 1710616943
	 anchor: eb38 [1.71061694e+09 3.75000000e+00] tt: 1710616943
dist: [0.4791 0.375 ]
3094 1710616944
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061694e+09 5.80800000e+00] tt: 1710616944
	 anchor: eb38 [1.71061694e+09 2.77200000e+00] tt: 1710616944
dist: [0.5808 0.2772]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061694e+09 -1.00000000e+00] tt: 1710616944
	 anchor: eb38 [ 1.71061694e+09 -1.00000000e+00] tt: 1710616944
dist: [-1. -1.]
3095 1710616945
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061694e+09 5.83100000e+00] tt: 1710616945
	 anchor: eb38 [1.71061694e+09 2.88000000e+00] tt: 1710616945
dist: [0.5831 0.288 ]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061694e+09 -1.00000000e+00] tt: 1710616945
	 anchor: eb38 [ 1.71061694e+09 -1.00000000e+00] tt: 1710616945
dist: [-1. -1.]
3096 1710616946
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.71061695e+09 5.72300000e+00] tt: 1710616

user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061702e+09 -1.00000000e+00] tt: 1710617016
	 anchor: eb38 [ 1.71061702e+09 -1.00000000e+00] tt: 1710617016
dist: [-1. -1.]
3167 1710617017
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [ 1.71061702e+09 -1.00000000e+00] tt: 1710617017
	 anchor: eb38 [ 1.71061702e+09 -1.00000000e+00] tt: 1710617017
dist: [-1. -1.]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061702e+09 -1.00000000e+00] tt: 1710617017
	 anchor: eb38 [ 1.71061702e+09 -1.00000000e+00] tt: 1710617017
dist: [-1. -1.]
3168 1710617018
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [ 1.71061702e+09 -1.00000000e+00] tt: 1710617018
	 anchor: eb38 [ 1.71061702e+09 -1.00000000e+00] tt: 1710617018
dist: [-1. -1.]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.71061702e+09 -1.00000000e+00] tt: 1710617018
	 anchor: eb38 [ 1.71061702e+09 -1.00000000e+00] tt: 1710617018
dist: [-1. -1.]
3169 1710617019
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [ 1.71061702e+09 -1.00000000e+00] tt: 1710617019


user f02e imgY (8, 8)
	 anchor: 8171 [1.7106171e+09 6.1450000e+00] tt: 1710617100
	 anchor: eb38 [1.7106171e+09 2.5380000e+00] tt: 1710617100
dist: [0.6145 0.2538]
user 867f imgY (8, 8)
	 anchor: 8171 [1.7106171e+09 5.2600000e+00] tt: 1710617100
	 anchor: eb38 [1.7106171e+09 3.7600000e+00] tt: 1710617100
dist: [0.526 0.376]
3251 1710617101
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.7106171e+09 6.0700000e+00] tt: 1710617101
	 anchor: eb38 [1.7106171e+09 2.1210000e+00] tt: 1710617101
dist: [0.607  0.2121]
user 867f imgY (8, 8)
	 anchor: 8171 [1.7106171e+09 5.2600000e+00] tt: 1710617101
	 anchor: eb38 [1.7106171e+09 3.7600000e+00] tt: 1710617101
dist: [0.526 0.376]
3252 1710617102
imgX (8, 8)
user f02e imgY (8, 8)
	 anchor: 8171 [1.7106171e+09 6.0460000e+00] tt: 1710617102
	 anchor: eb38 [1.7106171e+09 2.0880000e+00] tt: 1710617102
dist: [0.6046 0.2088]
user 867f imgY (8, 8)
	 anchor: 8171 [ 1.7106171e+09 -1.0000000e+00] tt: 1710617102
	 anchor: eb38 [ 1.7106171e+09 -1.0000000e+0

In [6]:
for user in users:
    XY0[user]=np.array(XY0[user])
    rssi0[user]=np.array(rssi0[user])
    Y0[user]=np.array(Y0[user])
T0=np.array(T0)
print(T0)
print(len(T0))
#print(XY0.shape)
#frame0=np.median(XY0, axis=(0)).astype(np.uint8)
#for i in range(0,XY0.shape[0]):
#    XY0[i]=XY0[i]-frame0
#print(XY0.shape)

[   0    1    2 ... 3342 3343 3344]
3345


In [7]:
minT=np.min(T0)
maxT=np.max(T0)

print(T0)
print(len(T0))
print(minT)
print(maxT)

print(T1)
print(len(T1))

[   0    1    2 ... 3342 3343 3344]
3345
0
3344
[1710613850, 1710613851, 1710613852, 1710613853, 1710613854, 1710613855, 1710613856, 1710613857, 1710613858, 1710613859, 1710613860, 1710613861, 1710613862, 1710613863, 1710613864, 1710613865, 1710613866, 1710613867, 1710613868, 1710613869, 1710613870, 1710613871, 1710613872, 1710613873, 1710613874, 1710613875, 1710613876, 1710613877, 1710613878, 1710613879, 1710613880, 1710613881, 1710613882, 1710613883, 1710613884, 1710613885, 1710613886, 1710613887, 1710613888, 1710613889, 1710613890, 1710613891, 1710613892, 1710613893, 1710613894, 1710613895, 1710613896, 1710613897, 1710613898, 1710613899, 1710613900, 1710613901, 1710613902, 1710613903, 1710613904, 1710613905, 1710613906, 1710613907, 1710613908, 1710613909, 1710613910, 1710613911, 1710613912, 1710613913, 1710613914, 1710613915, 1710613916, 1710613917, 1710613918, 1710613919, 1710613920, 1710613921, 1710613922, 1710613923, 1710613924, 1710613925, 1710613926, 1710613927, 1710613928, 171

In [8]:
XYT=[]
rssiT=[]
YT=[]

TA=int(window_size/2)
TB=int(window_size/2)

T=window_size

S=[]
for user in users:
    print("Usuario ",user)
    for t in range(minT+TA,maxT-TB):

        
        tt=T1[t]
        scene=getScene(tt)
        
        print("Scene",scene,tt,t)
        if not scene:
            continue

        subW=list(range(t-TA,t+TB))
        print("subW:",subW, "t:",t)
        if not (set(subW).intersection(set(T0)) == set(subW)):
            print("Not data for sliding window!",t)
            continue

        #print("rssi0[t-T:t+T]",rssi0[t-T:t+T].flatten())
        if(all(map(lambda x: x == -1, rssi0[user][t-TA:t].flatten()))):
            print("Distance -1 for RSSI in sliding window!",t)
            continue

        if(all(map(lambda x: x == -1, rssi0[user][t:t+TB].flatten()))):
            print("Distance -1 for RSSI in sliding window!",t)
            continue

        if(all(map(lambda x: x == 0, Y0[user][t].flatten()))):
            print("Distance 0 for Y0 in t!",t)
            continue


        if(XY0[user][t-TA:t+TB].shape[0]!=T):
            print("#",XY0[user][t-TA:t+TB].shape)
            continue
        if(rssi0[user][t-TA:t+TB].shape[0]!=T):
            print("@",rssi0[user][t-TA:t+TB].shape)
            continue

        XYT.append(XY0[user][t-TA:t+TB])
        rssiT.append(rssi0[user][t-TA:t+TB])
        YT.append(Y0[user][t])
        S.append(scene)
        #print(S)
#print(S)
XYT=np.array(XYT)
rssiT=np.array(rssiT)
YT=np.array(YT)    
S=np.array(S)    
print(XYT.shape)
print(rssiT.shape)
print(YT.shape)


Usuario  f02e
Scene Scene1 1710613852 2
subW: [0, 1, 2, 3] t: 2
Scene Scene1 1710613853 3
subW: [1, 2, 3, 4] t: 3
Scene Scene1 1710613854 4
subW: [2, 3, 4, 5] t: 4
Distance -1 for RSSI in sliding window! 4
Scene Scene1 1710613855 5
subW: [3, 4, 5, 6] t: 5
Distance -1 for RSSI in sliding window! 5
Scene Scene1 1710613856 6
subW: [4, 5, 6, 7] t: 6
Distance -1 for RSSI in sliding window! 6
Scene Scene1 1710613857 7
subW: [5, 6, 7, 8] t: 7
Distance -1 for RSSI in sliding window! 7
Scene Scene1 1710613858 8
subW: [6, 7, 8, 9] t: 8
Distance -1 for RSSI in sliding window! 8
Scene Scene1 1710613859 9
subW: [7, 8, 9, 10] t: 9
Distance -1 for RSSI in sliding window! 9
Scene Scene1 1710613860 10
subW: [8, 9, 10, 11] t: 10
Scene Scene1 1710613861 11
subW: [9, 10, 11, 12] t: 11
Distance -1 for RSSI in sliding window! 11
Scene Scene1 1710613862 12
subW: [10, 11, 12, 13] t: 12
Distance -1 for RSSI in sliding window! 12
Scene Scene1 1710613863 13
subW: [11, 12, 13, 14] t: 13
Distance -1 for RSSI in sl

Distance 0 for Y0 in t! 1386
Scene Scene3 1710615237 1387
subW: [1385, 1386, 1387, 1388] t: 1387
Distance 0 for Y0 in t! 1387
Scene Scene3 1710615238 1388
subW: [1386, 1387, 1388, 1389] t: 1388
Distance 0 for Y0 in t! 1388
Scene Scene3 1710615239 1389
subW: [1387, 1388, 1389, 1390] t: 1389
Distance 0 for Y0 in t! 1389
Scene Scene3 1710615240 1390
subW: [1388, 1389, 1390, 1391] t: 1390
Distance 0 for Y0 in t! 1390
Scene Scene3 1710615241 1391
subW: [1389, 1390, 1391, 1392] t: 1391
Distance 0 for Y0 in t! 1391
Scene Scene3 1710615242 1392
subW: [1390, 1391, 1392, 1393] t: 1392
Distance 0 for Y0 in t! 1392
Scene Scene3 1710615243 1393
subW: [1391, 1392, 1393, 1394] t: 1393
Distance 0 for Y0 in t! 1393
Scene Scene3 1710615244 1394
subW: [1392, 1393, 1394, 1395] t: 1394
Distance 0 for Y0 in t! 1394
Scene Scene3 1710615245 1395
subW: [1393, 1394, 1395, 1396] t: 1395
Distance 0 for Y0 in t! 1395
Scene Scene3 1710615246 1396
subW: [1394, 1395, 1396, 1397] t: 1396
Distance 0 for Y0 in t! 1396
S

Scene Scene3 1710615450 1600
subW: [1598, 1599, 1600, 1601] t: 1600
Distance -1 for RSSI in sliding window! 1600
Scene Scene3 1710615451 1601
subW: [1599, 1600, 1601, 1602] t: 1601
Scene Scene3 1710615452 1602
subW: [1600, 1601, 1602, 1603] t: 1602
Distance -1 for RSSI in sliding window! 1602
Scene Scene3 1710615453 1603
subW: [1601, 1602, 1603, 1604] t: 1603
Scene Scene3 1710615454 1604
subW: [1602, 1603, 1604, 1605] t: 1604
Scene Scene3 1710615455 1605
subW: [1603, 1604, 1605, 1606] t: 1605
Scene Scene3 1710615456 1606
subW: [1604, 1605, 1606, 1607] t: 1606
Scene Scene3 1710615457 1607
subW: [1605, 1606, 1607, 1608] t: 1607
Scene Scene3 1710615458 1608
subW: [1606, 1607, 1608, 1609] t: 1608
Scene Scene3 1710615459 1609
subW: [1607, 1608, 1609, 1610] t: 1609
Distance -1 for RSSI in sliding window! 1609
Scene Scene3 1710615460 1610
subW: [1608, 1609, 1610, 1611] t: 1610
Distance -1 for RSSI in sliding window! 1610
Scene Scene3 1710615461 1611
subW: [1609, 1610, 1611, 1612] t: 1611
Dist

Scene Scene4 1710616021 2171
subW: [2169, 2170, 2171, 2172] t: 2171
Distance -1 for RSSI in sliding window! 2171
Scene Scene4 1710616022 2172
subW: [2170, 2171, 2172, 2173] t: 2172
Scene Scene4 1710616023 2173
subW: [2171, 2172, 2173, 2174] t: 2173
Distance -1 for RSSI in sliding window! 2173
Scene Scene4 1710616024 2174
subW: [2172, 2173, 2174, 2175] t: 2174
Scene Scene4 1710616025 2175
subW: [2173, 2174, 2175, 2176] t: 2175
Scene Scene4 1710616026 2176
subW: [2174, 2175, 2176, 2177] t: 2176
Scene Scene4 1710616027 2177
subW: [2175, 2176, 2177, 2178] t: 2177
Distance -1 for RSSI in sliding window! 2177
Scene Scene4 1710616028 2178
subW: [2176, 2177, 2178, 2179] t: 2178
Scene Scene4 1710616029 2179
subW: [2177, 2178, 2179, 2180] t: 2179
Distance -1 for RSSI in sliding window! 2179
Scene Scene4 1710616030 2180
subW: [2178, 2179, 2180, 2181] t: 2180
Scene Scene4 1710616031 2181
subW: [2179, 2180, 2181, 2182] t: 2181
Scene Scene4 1710616032 2182
subW: [2180, 2181, 2182, 2183] t: 2182
Scen

Scene Scene1 1710613943 93
subW: [91, 92, 93, 94] t: 93
Distance -1 for RSSI in sliding window! 93
Scene Scene1 1710613944 94
subW: [92, 93, 94, 95] t: 94
Scene Scene1 1710613945 95
subW: [93, 94, 95, 96] t: 95
Distance -1 for RSSI in sliding window! 95
Scene Scene1 1710613946 96
subW: [94, 95, 96, 97] t: 96
Scene Scene1 1710613947 97
subW: [95, 96, 97, 98] t: 97
Distance -1 for RSSI in sliding window! 97
Scene Scene1 1710613948 98
subW: [96, 97, 98, 99] t: 98
Distance -1 for RSSI in sliding window! 98
Scene Scene1 1710613949 99
subW: [97, 98, 99, 100] t: 99
Distance -1 for RSSI in sliding window! 99
Scene Scene1 1710613950 100
subW: [98, 99, 100, 101] t: 100
Distance -1 for RSSI in sliding window! 100
Scene Scene1 1710613951 101
subW: [99, 100, 101, 102] t: 101
Distance -1 for RSSI in sliding window! 101
Scene Scene1 1710613952 102
subW: [100, 101, 102, 103] t: 102
Scene Scene1 1710613953 103
subW: [101, 102, 103, 104] t: 103
Distance -1 for RSSI in sliding window! 103
Scene Scene1 17

Scene Scene2 1710614566 716
subW: [714, 715, 716, 717] t: 716
Scene Scene2 1710614567 717
subW: [715, 716, 717, 718] t: 717
Scene Scene2 1710614568 718
subW: [716, 717, 718, 719] t: 718
Scene Scene2 1710614569 719
subW: [717, 718, 719, 720] t: 719
Scene Scene2 1710614570 720
subW: [718, 719, 720, 721] t: 720
Scene Scene2 1710614571 721
subW: [719, 720, 721, 722] t: 721
Scene Scene2 1710614572 722
subW: [720, 721, 722, 723] t: 722
Scene Scene2 1710614573 723
subW: [721, 722, 723, 724] t: 723
Scene Scene2 1710614574 724
subW: [722, 723, 724, 725] t: 724
Distance -1 for RSSI in sliding window! 724
Scene Scene2 1710614575 725
subW: [723, 724, 725, 726] t: 725
Distance -1 for RSSI in sliding window! 725
Scene Scene2 1710614576 726
subW: [724, 725, 726, 727] t: 726
Distance -1 for RSSI in sliding window! 726
Scene Scene2 1710614577 727
subW: [725, 726, 727, 728] t: 727
Distance -1 for RSSI in sliding window! 727
Scene Scene2 1710614578 728
subW: [726, 727, 728, 729] t: 728
Scene Scene2 17106

Scene Scene4 1710616065 2215
subW: [2213, 2214, 2215, 2216] t: 2215
Distance -1 for RSSI in sliding window! 2215
Scene Scene4 1710616066 2216
subW: [2214, 2215, 2216, 2217] t: 2216
Distance -1 for RSSI in sliding window! 2216
Scene Scene4 1710616067 2217
subW: [2215, 2216, 2217, 2218] t: 2217
Distance -1 for RSSI in sliding window! 2217
Scene Scene4 1710616068 2218
subW: [2216, 2217, 2218, 2219] t: 2218
Distance -1 for RSSI in sliding window! 2218
Scene Scene4 1710616069 2219
subW: [2217, 2218, 2219, 2220] t: 2219
Scene Scene4 1710616070 2220
subW: [2218, 2219, 2220, 2221] t: 2220
Distance -1 for RSSI in sliding window! 2220
Scene Scene4 1710616071 2221
subW: [2219, 2220, 2221, 2222] t: 2221
Distance -1 for RSSI in sliding window! 2221
Scene Scene4 1710616072 2222
subW: [2220, 2221, 2222, 2223] t: 2222
Distance -1 for RSSI in sliding window! 2222
Scene Scene4 1710616073 2223
subW: [2221, 2222, 2223, 2224] t: 2223
Distance -1 for RSSI in sliding window! 2223
Scene Scene4 1710616074 2224

Scene Scene6 1710617083 3233
subW: [3231, 3232, 3233, 3234] t: 3233
Distance -1 for RSSI in sliding window! 3233
Scene Scene6 1710617084 3234
subW: [3232, 3233, 3234, 3235] t: 3234
Distance -1 for RSSI in sliding window! 3234
Scene Scene6 1710617085 3235
subW: [3233, 3234, 3235, 3236] t: 3235
Distance -1 for RSSI in sliding window! 3235
Scene Scene6 1710617086 3236
subW: [3234, 3235, 3236, 3237] t: 3236
Distance -1 for RSSI in sliding window! 3236
Scene Scene6 1710617087 3237
subW: [3235, 3236, 3237, 3238] t: 3237
Scene Scene6 1710617088 3238
subW: [3236, 3237, 3238, 3239] t: 3238
Distance -1 for RSSI in sliding window! 3238
Scene Scene6 1710617089 3239
subW: [3237, 3238, 3239, 3240] t: 3239
Distance -1 for RSSI in sliding window! 3239
Scene Scene6 1710617090 3240
subW: [3238, 3239, 3240, 3241] t: 3240
Distance -1 for RSSI in sliding window! 3240
Scene Scene6 1710617091 3241
subW: [3239, 3240, 3241, 3242] t: 3241
Distance -1 for RSSI in sliding window! 3241
Scene Scene6 1710617092 3242

In [9]:
import matplotlib.pyplot as plt

L=20

# Create a figure with two subplots (one row, two columns)
fig, axes = plt.subplots(L, 2, figsize=(200, 200))


for k in range(0,L):
    mpAll=XY0[users[0]][k].reshape(w,h)

    axes[k,0].imshow(mpAll, cmap='viridis', interpolation='nearest')
    axes[k,0].set_title('All players (last frame)')
    axes[k,0].axis('off')

    mpOne=Y0[users[0]][k].reshape(w,h)

    axes[k,1].imshow(mpOne, cmap='viridis', interpolation='nearest')
    axes[k,1].set_title('True (last frame) of player')
    axes[k,1].axis('off')

    
# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

In [10]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Softmax,Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D
from tensorflow.keras.models import Model

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from keras.losses import mean_squared_error

from keras import backend as K

from sklearn.model_selection import KFold

from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Lambda, Dot,Reshape,Multiply, Input, Conv1D, Conv2D, ConvLSTM2D,ReLU,UpSampling2D, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D, Concatenate
from tensorflow.keras.models import Model, Sequential

from keras import backend as K
import math

from tensorflow.keras.losses import CategoricalCrossentropy

import tensorflow as tf
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.layers import Conv2DTranspose

In [11]:
#MODELO A

K=len(anchors)
print(K)
print(T)
rssi = Input(shape=(T,K,1))
print(rssi)

encoded_rssi = TimeDistributed(Conv1D(32, 3, activation='relu', padding='same',
        input_shape=(T, K, 1)))(rssi)
#encoded_rssi = TimeDistributed(MaxPooling1D(2, padding='same'))(encoded_rssi)
encoded_rssi = TimeDistributed(Conv1D(64, 3, activation='relu', padding='same'))(encoded_rssi)
print(encoded_rssi)
encoded_rssi = TimeDistributed(Flatten())(encoded_rssi)

encoded_rssi=LSTM(128, return_sequences=True)(encoded_rssi)
encoded_rssi=LSTM(128, return_sequences=True)(encoded_rssi)
print("lstm",encoded_rssi)
encoded_rssi=Reshape((T,1,1,128))(encoded_rssi)

print("lstm",encoded_rssi)

N=w
M=h

xy = Input(shape=(T,N,M,1))
print(xy)

encoded = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'),
        input_shape=(T, N, M, 1))(xy)
encoded = TimeDistributed(MaxPooling2D((2, 2), padding='same'))(encoded)
encoded = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(encoded)
#encoded = TimeDistributed(MaxPooling2D((2, 2), padding='same'))(encoded)
#encoded = TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same'))(encoded)
#encoded = TimeDistributed(MaxPooling2D((2, 2), padding='same'))(encoded)
#encoded = TimeDistributed(Conv2D(256, (3, 3), activation='relu', padding='same'))(encoded)
print(encoded)

#LayerNormalization
encoded= ConvLSTM2D(128 , 3,return_sequences=True, padding='same')(encoded)

encoded= ConvLSTM2D(128 , 3,return_sequences=True, padding='same')(encoded)


print(encoded)

#merged=Multiply()([encoded, encoded_rssi])
encoded_rssi2=Reshape((4,128))(encoded_rssi)
print(encoded_rssi2)
print(encoded)
merged=Dot(axes=[4, 2])([encoded, encoded_rssi2])
merged=Softmax()(merged)
# Create a lambda layer to expand dimensions of input2
#expand_dim = Lambda(lambda x: tf.repeat(x, 8, axis=3))(encoded_rssi)  # Repeat along axis 2
#merged=Concatenate(axis=2)([encoded, encoded_rssi])

print(merged)

decoded= ConvLSTM2D(128 , 3, padding='same')(merged)

#decoded = (Conv2D(256, (3, 3), activation='relu', padding='same'))(decoded)
#decoded = (UpSampling2D((2, 2)))(decoded)
#decoded = (Conv2D(128, (3, 3), activation='relu', padding='same'))(decoded)
#decoded = (UpSampling2D((2, 2)))(decoded)
decoded = (Conv2D(64, (3, 3), activation='relu', padding='same'))(decoded)
decoded = (UpSampling2D((2, 2)))(decoded)
decoded = (Conv2D(32, (3, 3), activation='relu', padding='same'))(decoded)
decoded = (Conv2D(1, (3, 3), activation='sigmoid', padding='same'))(decoded)

# Compile model
autoencoder = Model([xy,rssi], decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse', 'mae'])
#autoencoder.compile(optimizer=Adam(learning_rate=0.001),
#              loss=CategoricalCrossentropy(),
#              metrics=['accuracy'])

print(encoded)
autoencoder.save("model0.h5")
autoencoder.summary()

2
4
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 2, 1), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 2, 64), dtype=tf.float32, name=None), name='time_distributed_1/Reshape_1:0', description="created by layer 'time_distributed_1'")
lstm KerasTensor(type_spec=TensorSpec(shape=(None, 4, 128), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:1', description="created by layer 'lstm_1'")
lstm KerasTensor(type_spec=TensorSpec(shape=(None, 4, 1, 1, 128), dtype=tf.float32, name=None), name='reshape/Reshape:0', description="created by layer 'reshape'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 8, 8, 1), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4, 4, 64), dtype=tf.float32, name=None), name='time_distributed_5/Reshape_1:0', description="created by layer 'time_distributed_5'")
KerasT

In [12]:
#MODELO SOLO LSTM

# Rama secuencial
K=len(anchors)

rssi_input = Input(shape=(T, K, 1))
rssi_flattened = TimeDistributed(Flatten())(rssi_input)
rssi_lstm = LSTM(units=128, return_sequences=True)(rssi_flattened)

N=w
M=h

xy_input = Input(shape=(T, N, M, 1))
xy_flattened = TimeDistributed(Flatten())(xy_input)
xy_lstm1 = LSTM(units=256, return_sequences=True)(xy_flattened)  # Mayor capacidad
xy_lstm2 = LSTM(units=128, return_sequences=True)(xy_lstm1)  # Segunda capa LSTM

print(xy_lstm2)

# Aplastando las salidas para que puedan ser concatenadas
rssi_lstm_flattened = Flatten()(rssi_lstm)
xy_lstm_flattened = Flatten()(xy_lstm2)

# Fusión de características

merged = Concatenate()([rssi_lstm_flattened, xy_lstm_flattened])
dense_output = Dense(units=8*8, activation='sigmoid')(merged)
output = Reshape((8, 8, 1))(dense_output)

# Modelo
model = Model(inputs=[xy_input, rssi_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse', 'mae'])

model.save("model1.h5")
model.summary()



KerasTensor(type_spec=TensorSpec(shape=(None, 4, 128), dtype=tf.float32, name=None), name='lstm_4/PartitionedCall:1', description="created by layer 'lstm_4'")
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 4, 8, 8, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 input_3 (InputLayer)           [(None, 4, 2, 1)]    0           []                               
                                                                                                  
 time_distributed_7 (TimeDistri  (None, 4, 64)       0           ['input_4[0][0]']                
 buted)                         

In [13]:
#MODELO LIGERO: LSTM+CONV+LSTM

# Rama secuencial
K=len(anchors)
print(K)

rssi = Input(shape=(T,K,1))
print(rssi)
lstm1=TimeDistributed(Flatten())(rssi)

lstm_out = LSTM(units=128)(lstm1)
print("LSTM",lstm_out)

N=w
M=h

print(w,h)

xy = Input(shape=(T,N,M,1))
print(xy)

conv1 = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(xy)
conv2 = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(conv1)
conv_out = TimeDistributed(Flatten())(conv2)
print("CONV",conv_out)

lstm2= LSTM(units=1024,return_sequences=True)(conv_out)
lstm2_out=LSTM(units=128,return_sequences=True)(lstm2)
lstm2_out=Flatten()(lstm2_out)
print(lstm2_out)

# Fusión de características

merged = Concatenate()([lstm_out, lstm2_out])
dense1 = Dense(128, activation='relu')(merged)
reshape = Reshape((2, 2, 32))(dense1) 


# Deconvolución para generar imagen
conv2dtranspose1 = Conv2DTranspose(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(reshape)
conv2dtranspose2 = Conv2DTranspose(filters=1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='sigmoid')(conv2dtranspose1)

# Modelo
model = Model(inputs=[xy, rssi], outputs=conv2dtranspose2)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse', 'mae']) 

model.save("model2.h5")
model.summary()

2
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 2, 1), dtype=tf.float32, name='input_5'), name='input_5', description="created by layer 'input_5'")
LSTM KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='lstm_5/PartitionedCall:0', description="created by layer 'lstm_5'")
8 8
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 8, 8, 1), dtype=tf.float32, name='input_6'), name='input_6', description="created by layer 'input_6'")
CONV KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4096), dtype=tf.float32, name=None), name='time_distributed_9/Reshape_1:0', description="created by layer 'time_distributed_9'")
KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='flatten_7/Reshape:0', description="created by layer 'flatten_7'")
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to  

In [14]:
#MODELO LIGERO2: LSTM+CONV+LSTM con DROPOUT Y BATCHNORMALIZATION

# Rama secuencial
K=len(anchors)
print(K)

rssi = Input(shape=(T,K,1))
print(rssi)
lstm1=TimeDistributed(Flatten())(rssi)

lstm_out = LSTM(units=128,dropout=0.2)(lstm1)
print("LSTM",lstm_out)

N=w
M=h

print(w,h)

xy = Input(shape=(T,N,M,1))
print(xy)

conv1 = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(xy)
conv1 = BatchNormalization()(conv1)  # Añadido BatchNormalization
conv2 = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(conv1)
conv2 = BatchNormalization()(conv2)  # Añadido BatchNormalization
conv_out = TimeDistributed(Flatten())(conv2)
print("CONV",conv_out)

lstm2= LSTM(units=1024,return_sequences=True,dropout=0.2)(conv_out)
lstm2_out=LSTM(units=128,return_sequences=True,dropout=0.2)(lstm2)
lstm2_out=Flatten()(lstm2_out)
print(lstm2_out)

# Fusión de características

merged = Concatenate()([lstm_out, lstm2_out])
dense1 = Dense(128, activation='relu')(merged)
reshape = Reshape((2, 2, 32))(dense1) 


# Deconvolución para generar imagen
conv2dtranspose1 = Conv2DTranspose(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(reshape)
conv2dtranspose2 = Conv2DTranspose(filters=1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='sigmoid')(conv2dtranspose1)

# Modelo
modelA2 = Model(inputs=[xy, rssi], outputs=conv2dtranspose2)
modelA2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse', 'mae']) 

modelA2.save("model3.h5")
modelA2.summary()

2
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 2, 1), dtype=tf.float32, name='input_7'), name='input_7', description="created by layer 'input_7'")
LSTM KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='lstm_8/PartitionedCall:0', description="created by layer 'lstm_8'")
8 8
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 8, 8, 1), dtype=tf.float32, name='input_8'), name='input_8', description="created by layer 'input_8'")
CONV KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4096), dtype=tf.float32, name=None), name='time_distributed_11/Reshape_1:0', description="created by layer 'time_distributed_11'")
KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='flatten_10/Reshape:0', description="created by layer 'flatten_10'")
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected 

In [15]:
#MODELO CON ATTENTIONAL DE 128

from keras.layers import Input, LSTM, Dense, Concatenate, Reshape, Conv2D, Conv2DTranspose, TimeDistributed, Flatten, BatchNormalization, Dropout, Layer
import keras.backend as KE
from tensorflow.keras.utils import register_keras_serializable

@register_keras_serializable()
class Attention(layers.Layer):
    def __init__(self, units=128, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, features, hidden):
        # hidden shape == (batch_size, hidden size)
        hidden_with_time_axis = KE.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, hidden size)
        score = KE.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = KE.softmax(self.V(score), axis=1)
        
        # context_vector shape after sum == (batch_size, hidden size)
        context_vector = attention_weights * features
        context_vector = KE.sum(context_vector, axis=1)

        return context_vector, attention_weights

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

    def get_config(self):
        base_config = super(Attention, self).get_config()
        return base_config
    
    
# Rama secuencial
K=len(anchors)
print(K)

rssi = Input(shape=(T,K,1))
print(rssi)
lstm1=TimeDistributed(Flatten())(rssi)

lstm_out = LSTM(units=128,dropout=0.2)(lstm1)
print("LSTM",lstm_out)

N=w
M=h

print(w,h)

xy = Input(shape=(T,N,M,1))
print(xy)

conv1 = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(xy)
conv1 = BatchNormalization()(conv1)  # Añadido BatchNormalization
conv2 = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(conv1)
conv2 = BatchNormalization()(conv2)  # Añadido BatchNormalization
conv_out = TimeDistributed(Flatten())(conv2)
print("CONV",conv_out)

lstm2= LSTM(units=1024,return_sequences=True,dropout=0.2)(conv_out)
lstm2= LSTM(units=128,return_sequences=True,dropout=0.2)(lstm2)
lstm2_out=LSTM(units=32,return_sequences=True,dropout=0.2)(lstm2)
lstm2_out=Flatten()(lstm2_out)
print("LSTM2",lstm2_out)

# Aplicando atención a la salida de la primera LSTM
attention_layer = Attention(128)
context_vector1, attention_weights1 = attention_layer(lstm_out, lstm_out)

# Aplicando atención a la salida de la segunda LSTM
context_vector2, attention_weights2 = attention_layer(lstm2_out, lstm2_out)

# Fusión de características con vectores de contexto de capas de atención
merged = Concatenate()([context_vector1, context_vector2])
dense1 = Dense(128, activation='relu')(merged)
reshape = Reshape((2, 2, 32))(dense1)

# Deconvolución para generar imagen
conv2dtranspose1 = Conv2DTranspose(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(reshape)
conv2dtranspose2 = Conv2DTranspose(filters=1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='sigmoid')(conv2dtranspose1)

# Modelo
modelA2 = Model(inputs=[xy, rssi], outputs=conv2dtranspose2)
modelA2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse', 'mae']) 
modelA2.save("model4.h5")
modelA2.summary()


2
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 2, 1), dtype=tf.float32, name='input_9'), name='input_9', description="created by layer 'input_9'")
LSTM KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='lstm_11/PartitionedCall:0', description="created by layer 'lstm_11'")
8 8
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 8, 8, 1), dtype=tf.float32, name='input_10'), name='input_10', description="created by layer 'input_10'")
CONV KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4096), dtype=tf.float32, name=None), name='time_distributed_13/Reshape_1:0', description="created by layer 'time_distributed_13'")
LSTM2 KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='flatten_13/Reshape:0', description="created by layer 'flatten_13'")
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #    

In [16]:
from keras.layers import Multiply

def apply_attention_2d(inputs):
    # Genera mapas de atención usando una convolución
    attention_map = Conv2D(filters=1, kernel_size=(3, 3), padding='same', activation='sigmoid')(inputs)
    
    # Aplica el mapa de atención a los inputs (element-wise multiplication)
    attended_inputs = Multiply()([inputs, attention_map])
    
    return attended_inputs


# Rama secuencial
K=len(anchors)
print(K)

rssi = Input(shape=(T,K,1))
print(rssi)
lstm1=TimeDistributed(Flatten())(rssi)

lstm_out = LSTM(units=128,dropout=0.2)(lstm1)
print("LSTM",lstm_out)

N=w
M=h

print(w,h)

xy = Input(shape=(T,N,M,1))
print(xy)

conv1 = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(xy)
# Aplicando atención 2D después de las convoluciones en tu modelo
conv1_attended = apply_attention_2d(conv1)
conv2 = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(conv1_attended)
conv2_attended = apply_attention_2d(conv2)
conv2 = BatchNormalization()(conv2_attended) 
conv_out = TimeDistributed(Flatten())(conv2)
print("CONV",conv_out)

lstm2= LSTM(units=1024,return_sequences=True,dropout=0.2)(conv_out)
lstm2_out=LSTM(units=128,return_sequences=True,dropout=0.2)(lstm2)
lstm2_out=Flatten()(lstm2_out)
print(lstm2_out)

# Fusión de características

merged = Concatenate()([lstm_out, lstm2_out])
dense1 = Dense(128, activation='relu')(merged)
reshape = Reshape((2, 2, 32))(dense1) 


# Deconvolución para generar imagen
conv2dtranspose1 = Conv2DTranspose(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(reshape)
conv2dtranspose2 = Conv2DTranspose(filters=1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='sigmoid')(conv2dtranspose1)

# Modelo
modelA2 = Model(inputs=[xy, rssi], outputs=conv2dtranspose2)
modelA2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse', 'mae']) 

modelA2.save("model5.h5")
modelA2.summary()

2
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 2, 1), dtype=tf.float32, name='input_11'), name='input_11', description="created by layer 'input_11'")
LSTM KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='lstm_15/PartitionedCall:0', description="created by layer 'lstm_15'")
8 8
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 8, 8, 1), dtype=tf.float32, name='input_12'), name='input_12', description="created by layer 'input_12'")
CONV KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4096), dtype=tf.float32, name=None), name='time_distributed_15/Reshape_1:0', description="created by layer 'time_distributed_15'")
KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='flatten_16/Reshape:0', description="created by layer 'flatten_16'")
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Co

In [17]:
#MODELO CON ATTENTIONAL QUE TIENE EN CUENTA PESOS  
    
# Rama secuencial
K=len(anchors)
print(K)

rssi = Input(shape=(T,K,1))
print(rssi)
lstm1=TimeDistributed(Flatten())(rssi)

lstm_out = LSTM(units=128,dropout=0.2)(lstm1)
print("LSTM",lstm_out)

N=w
M=h

print(w,h)

xy = Input(shape=(T,N,M,1))
print(xy)

conv1 = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(xy)
conv1 = BatchNormalization()(conv1)  # Añadido BatchNormalization
conv2 = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(conv1)
conv2 = BatchNormalization()(conv2)  # Añadido BatchNormalization
conv_out = TimeDistributed(Flatten())(conv2)
print("CONV",conv_out)

lstm2= LSTM(units=1024,return_sequences=True,dropout=0.2)(conv_out)
lstm2= LSTM(units=128,return_sequences=True,dropout=0.2)(lstm2)
lstm2_out=LSTM(units=32,return_sequences=True,dropout=0.2)(lstm2)
lstm2_out=Flatten()(lstm2_out)
print("LSTM2",lstm2_out)


# Primero, aplicamos la capa de atención a la salida de lstm_out_128
attention_layer_128 = Attention(128)
context_vector_128, attention_weights_128 = attention_layer_128(lstm_out, lstm_out)


# Aplicando atención a la salida de la segunda LSTM
attention_layer_512 = Attention(512)
context_vector_512, attention_weights_512 = attention_layer_512(lstm2_out, lstm2_out)

# Fusión de características con vectores de contexto de capas de atención
merged = Concatenate()([context_vector_128, context_vector_512])
dense1 = Dense(128, activation='relu')(merged)
reshape = Reshape((2, 2, 32))(dense1)

# Deconvolución para generar imagen
conv2dtranspose1 = Conv2DTranspose(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(reshape)
conv2dtranspose2 = Conv2DTranspose(filters=1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='sigmoid')(conv2dtranspose1)

# Modelo
modelA2 = Model(inputs=[xy, rssi], outputs=conv2dtranspose2)
modelA2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse', 'mae']) 

modelA2.save("model6.h5")
modelA2.summary()


2
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 2, 1), dtype=tf.float32, name='input_13'), name='input_13', description="created by layer 'input_13'")
LSTM KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='lstm_18/PartitionedCall:0', description="created by layer 'lstm_18'")
8 8
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 8, 8, 1), dtype=tf.float32, name='input_14'), name='input_14', description="created by layer 'input_14'")
CONV KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4096), dtype=tf.float32, name=None), name='time_distributed_17/Reshape_1:0', description="created by layer 'time_distributed_17'")
LSTM2 KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='flatten_19/Reshape:0', description="created by layer 'flatten_19'")
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param # 

In [18]:
def plotting(XYT_test,YT_test,YT_prediction):
    K=5
    off=3

    # Create a figure with two subplots (one row, two columns)
    fig, axes = plt.subplots(K, 3, figsize=(20, 20))

    for k in range(0,K):
        mpAll=XYT_test[k+off][T-1].reshape(N,M)

        axes[k,0].imshow(mpAll, cmap='viridis', interpolation='nearest')
        axes[k,0].set_title('All players (last frame)')
        axes[k,0].axis('off')

        label=rssiT_test[k+off][T-1]
        mpOne=YT_test[k+off].reshape(N,M)

        axes[k,1].imshow(mpOne, cmap='viridis', interpolation='nearest')
        axes[k,1].set_title('True (last frame) of player')
        axes[k,1].axis('off')


        mpPrediction=YT_prediction[k+off].reshape(N,M)

        axes[k,2].imshow(mpPrediction, cmap='viridis', interpolation='nearest')
        axes[k,2].set_title('Prediction of player ')
        axes[k,2].axis('off')


    # Adjust spacing between subplots
    plt.tight_layout()

    # Show the plots
    plt.show()

In [19]:
def plottingRandom(XYT_test,YT_test,YT_prediction):
    K=5
    ks=np.random.randint(0, len(test_indices), K)

    # Create a figure with two subplots (one row, two columns)
    fig, axes = plt.subplots(K, 3, figsize=(20, 20))

    for k in range(0,K):
        mpAll=XYT_test[ks[k]][T-1].reshape(N,M)

        axes[k,0].imshow(mpAll, cmap='viridis', interpolation='nearest')
        axes[k,0].set_title('All players (last frame)')
        axes[k,0].axis('off')

        label=rssiT_test[ks[k]][T-1]
        mpOne=YT_test[ks[k]].reshape(N,M)

        axes[k,1].imshow(mpOne, cmap='viridis', interpolation='nearest')
        axes[k,1].set_title('True (last frame) of player')
        axes[k,1].axis('off')


        mpPrediction=YT_prediction[ks[k]].reshape(N,M)

        axes[k,2].imshow(mpPrediction, cmap='viridis', interpolation='nearest')
        axes[k,2].set_title('Prediction of player ')
        axes[k,2].axis('off')


    # Adjust spacing between subplots
    plt.tight_layout()

    # Show the plots
    plt.show()

In [ ]:
import numpy as np
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error

np.random.seed(42)

# Lista de identificadores de modelo
model_ids = ["model0","model1","model2","model4"] 

# Estructuras para almacenar resultados y tiempos de ejecución
mse_results = {model_id: [] for model_id in model_ids}
mae_results = {model_id: [] for model_id in model_ids}
training_times = {model_id: [] for model_id in model_ids}
total_execution_time = {model_id: 0 for model_id in model_ids}

print(np.unique(S))

for ixs, sc in enumerate(np.unique(S)):
    # Índices para entrenamiento y prueba
    train_indices = np.where(S != sc)[0]
    test_indices = np.where(S == sc)[0]
    np.random.shuffle(train_indices)

    XYT_train = XYT[train_indices]
    rssiT_train = rssiT[train_indices]
    YT_train = YT[train_indices]

    XYT_test = XYT[test_indices]
    rssiT_test = rssiT[test_indices]
    YT_test = YT[test_indices]
    
    for model_id in model_ids:
        print(f"MODELO {model_id} - ESCENA {sc}")
        
        model = load_model(f"{model_id}.h5")

        start_time = time.time()

        history = model.fit([XYT_train, rssiT_train], YT_train,
                                  batch_size=16,
                                  validation_data=([XYT_test, rssiT_test], YT_test),
                                  epochs=50)
        end_time = time.time()
        training_time = end_time - start_time
        
        YT_prediction = model.predict([XYT_test, rssiT_test])

        plottingRandom(XYT_test, YT_test, YT_prediction)

        mse = mean_squared_error(YT_test.flatten(), YT_prediction.flatten())
        mae = mean_absolute_error(YT_test.flatten(), YT_prediction.flatten())

        mse_results[model_id].append(mse)
        mae_results[model_id].append(mae)
        training_times[model_id].append(training_time)
        total_execution_time[model_id] += training_time

# Calculando promedios y mostrando resultados

print("Resultados por escena:")
for i, sc in enumerate(np.unique(S)):
    print(f"Escena {sc}:")
    for model_id in model_ids:
        print(f"Modelo {model_id} - MSE: {mse_results[model_id][i]:.4f}, MAE: {mae_results[model_id][i]:.4f}, Tiempo: {training_times[model_id][i]:.2f}s")
    print("")
    
print("RESUMEN RESULTADOS:")
for model_id in model_ids:
    average_mse = np.mean(mse_results[model_id])
    average_mae = np.mean(mae_results[model_id])
    print(f"Modelo {model_id}:")
    print(f"Tiempo de ejecución total: {total_execution_time[model_id]:.2f} segundos")
    print(f"MSE promedio: {average_mse:.4f}")
    print(f"MAE promedio: {average_mae:.4f}\n")


['Scene1' 'Scene2' 'Scene3' 'Scene4' 'Scene5' 'Scene6']
MODELO model0 - ESCENA Scene1
Epoch 1/50
100/100 [==============================] - 17s 119ms/step - loss: 0.1357 - mse: 0.0265 - mae: 0.0638 - val_loss: 0.0934 - val_mse: 0.0155 - val_mae: 0.0320
Epoch 2/50
100/100 [==============================] - 11s 108ms/step - loss: 0.0882 - mse: 0.0163 - mae: 0.0358 - val_loss: 0.0914 - val_mse: 0.0155 - val_mae: 0.0344
Epoch 3/50
100/100 [==============================] - 11s 111ms/step - loss: 0.0874 - mse: 0.0162 - mae: 0.0361 - val_loss: 0.0913 - val_mse: 0.0155 - val_mae: 0.0332
Epoch 4/50
100/100 [==============================] - 11s 112ms/step - loss: 0.0868 - mse: 0.0162 - mae: 0.0355 - val_loss: 0.0943 - val_mse: 0.0158 - val_mae: 0.0434
Epoch 5/50
100/100 [==============================] - 11s 111ms/step - loss: 0.0868 - mse: 0.0162 - mae: 0.0358 - val_loss: 0.0902 - val_mse: 0.0155 - val_mae: 0.0354
Epoch 6/50
100/100 [==============================] - 11s 109ms/step - loss: 0.

In [ ]:
for it,tt in enumerate(ts):
    print(it,tt)